In [ ]:
from datetime import datetime
from pydantic import BaseModel, PositiveInt,field_validator


class User(BaseModel):
    id: int  
    name: str = 'John Doe'  
    signup_ts: datetime | None  
    gender: list
    tastes: dict[str, PositiveInt]
    attributes_l: list[str]
    
    @field_validator('attributes_l',mode='before' )
    def validate_each_string(cls, value):
        #if not isinstance(value, list):
        #    raise ValueError('strings must be a list of strings')
        
        for item in value:
            if not isinstance(item, str):
                raise ValueError('All items must be strings')
        return value

external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',  
    'gender': [22,33,444],
    'tastes': {
        'wine': 9,
        b'cheese': 7,  
        'cabbage': '1',  
    },
    'attributes_l': ["Region","Item","Year"]
}

user = User(**external_data)  

print(user.id)  
#> 123
print(user.model_dump())  


In [ ]:
from pydantic import BaseModel, Field, ValidationError, field_validator
from typing import List

class MyModel(BaseModel):
    strings: List[str]
    
    # Using field_validator for field-level validation
    @field_validator('strings', mode='before')
    def validate_each_string(cls, value):
        if not isinstance(value, list):
            raise ValueError('strings must be a list of strings')
        
        for item in value:
            if not isinstance(item, str):
                raise ValueError('All items must be strings')
        return value

# Example usage
try:
    data = MyModel(strings=["hello", "world", "py"])
    data = MyModel(strings=["123", "4433", "4456"])
    print(data)
except ValidationError as e:
    print(e.json())

In [ ]:
from mstr_robotics._connectors import mstr_api
from mstr_robotics.report import rep
from mstrio.connection import Connection
import pandas as pd
import json

base_url="http://85.214.60.83:8080/MicroStrategyLibrary"
path="D:\\shared_drive\\Python\\mstr_robotics\\mstr_robotics\\user_d.json"

with open(path, 'r') as file:
    user_d = json.load(file)

username=user_d["username"]
password=user_d["password"]

i_rep=rep()

project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"

conn = Connection(base_url=base_url,username=username,password=password,project_id=project_id)
conn.headers['Content-type'] = "application/json"

In [ ]:
report_id="E247680C45150529BB48CF9743C9C69F"
dp=i_rep.get_default_prp_answ(conn=conn, report_id=report_id)

In [ ]:
dp.json()

In [ ]:
rep_d={'template': {'metrics': ['Revenue', 'Cost', 'Profit'],
  'attributes': ['Category', 'Call Center']},
 'filter': {"year": {"In": [2021, 2022]}, "revenue": {"Greater": 200; "Level":[]}, "region": {"BeginsWith": "S"}}

In [ ]:
filter_d = {"year": {"In": [2021, 2022]}, "revenue": {"Greater": 200; "Level":[]}, "region": {"BeginsWith": "S"}}
#filter_d = {"year": {"In": [2021, 2022]} and "revenue": {"Percent.Top": 10,"level":["Customers"]} and "region": {"BeginsWith": "S"}}

In [5]:
from pydantic import BaseModel, Field, ValidationError,RootModel
from typing import List, Dict, Union,Literal,Optional

class objects(BaseModel):
    metrics: List[Literal["Revenue", "Cost", "Profit"]]
    attributes: List[Literal["Category","Call Center","Region","Year"]]

class TemplateModel(BaseModel):
    metrics: List[str]
    attributes: List[str]

class operators(BaseModel):
    In: List[Union[int, str]] = Field(default=[])
    Greater: Union[int, None] = Field(default=None)
    Level: List[Union[int, str]] = Field(default=[])
    BeginsWith: Union[str, None] = Field(default=None)
    
class FilterModel(BaseModel):
    RootModel: Optional[Dict]  # Accept any key with a FilterOperator as the value
    In: List[Union[int, str]] = Field(default=[])


class MainModel(BaseModel):
    template: Optional[TemplateModel]
    filter: Optional[FilterModel]


# Example usage
data = {
    'template': {
        'metrics': ['Revenue', 'Cost', 'Profit'],
        'attributes': ['Category', 'Call Center',"Year"]
    },
    'filter': {
        "In": [2021, 2022],
        "BeginsWith": "S"
                }
    
}

try:
    model = MainModel(**data)
    print(model)
except ValidationError as e:
    print(e.json())

[{"type":"missing","loc":["filter","RootModel"],"msg":"Field required","input":{"In":[2021,2022],"BeginsWith":"S"},"url":"https://errors.pydantic.dev/2.9/v/missing"}]


In [ ]:
from pydantic import BaseModel
from typing import Dict
from typing import Union, List, Dict, Optional

class FilterCondition(BaseModel):
    In: list[int | str] = []
    Greater: int | None = None
    #Level: Dict[str,List[str |int]] ={}
    Level: Optional[Dict[str, List[Union[str, int]]]] = Field(default_factory=dict)

    BeginsWith: str | None = None

class DynamicFilterModel(BaseModel):
    filters: Dict[str, FilterCondition]  # Key: dynamic attribute/metric name, Value: FilterCondition

# Example Usage
data = {
    "filters": {
        "year": {"In": [2021, 2022], "Greater": None, "Level": None, "BeginsWith": None},
        "revenue": {"Greater": 200, "Level": { "attribute_l":[555,2122]}},
        "region": {"BeginsWith": "S"},
        #"profit": {"Greater": 500, "Level": {"report":""}}
    }
}

# Validate the data
try:
    dynamic_filter = DynamicFilterModel(**data)
    print(dynamic_filter)
except ValidationError as e:
    print(e.json())